In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_train = pd.read_csv("../input/tabular-playground-series-may-2022/train.csv")
df_test = pd.read_csv("../input/tabular-playground-series-may-2022/test.csv")
id1 = df_train["id"]
id2 = df_test["id"]
df_train.drop(columns=["id"], inplace=True)
df_test.drop(columns=["id"], inplace=True)

In [ ]:
from sklearn.model_selection import StratifiedKFold
sfk = StratifiedKFold()
def folds(df, set_):
    df["kfold"] = -1
    if set_ == "train":
        df["id"] = id1
        for f , (tid, vid) in enumerate(sfk.split(df, df["target"])):
            df.loc[vid, "kfold"] = f
    if set_ == "test":
        df["id"] = id2
        df
    return df

In [ ]:
train_folds = folds(df_train, "train")
test = folds(df_test, "test")

In [ ]:
test

In [ ]:
def reindex_df(df, excluding_columns):
    id_ = df.iloc[:, -excluding_columns:]
    cat = df.loc[:, df.dtypes=="int64"].iloc[:, :-excluding_columns]
    cont = df.loc[:, df.dtypes=="float64"]
    f27 = df.loc[:, df.dtypes=="object"]
    return pd.concat([id_,cat, cont, f27], axis=1)

In [ ]:
train_folds = reindex_df(train_folds, 3)
test = reindex_df(test,2)

In [ ]:
# alphabet dictionary
import string
i = 0
range_ = list(range(1,27))
alphabets = dict.fromkeys(string.ascii_uppercase,)
for name, key in alphabets.items():
    alphabets[name] = range_[i]
    i = i + 1

def f27_features(f27):
    features_split = []
    for id_ in f27.index:
        string = list(f27.loc[id_,"f_27"])
        features_split.append(string)

    encoded_features = []
    for feature in features_split:
        encoded = []
        for char in feature:
            encoded.append(alphabets[char])
        encoded_features.append(encoded)

    col_names = []
    for i in range(10):
        name = "f27_" + str(i)
        col_names.append(name)
    new_features = pd.DataFrame(encoded_features, columns=col_names)
    return new_features

In [ ]:
new_features = f27_features(train_folds)
train_folds = pd.concat([train_folds, new_features], axis=1)
train_folds.drop(columns=["f_27"], inplace=True)

In [ ]:
import lightgbm
from sklearn.metrics import accuracy_score
def objective(trial, df):
    params = {
    'objective':'binary',
    'metric' : 'binary_error',
    'boosting': 'gbdt',
    'num_leaves': trial.suggest_int("num_leaves",2 , 256),
    'feature_fraction': trial.suggest_float("feature_fraction", 0.1, 1.0),
    'bagging_fraction': trial.suggest_float("bagging_fraction", 0.1, 1.0),
    'learning_rate': trial.suggest_float("learning_rate", 0.01,1) }
    accuracy = []
    for fold in range(5):
        train = df[df["kfold"] != fold]
        valid = df[df["kfold"] == fold]
        train_lgbm = lightgbm.Dataset(train.iloc[:,3:].to_numpy(), train["target"].to_numpy())
        model_lgbm = lightgbm.train(params, train_lgbm, num_boost_round=200)
        pred_prob = model_lgbm.predict(valid.iloc[:,3:].to_numpy())
        y_pred = []
        for pred in pred_prob:
            if pred >= 0.5:
                y_pred.append(1)
            else:
                y_pred.append(0)
        ypred = np.array(y_pred)
        fold_accuracy = accuracy_score(valid["target"].to_numpy(), ypred)
        accuracy.append(fold_accuracy)
        return np.mean(accuracy)

In [ ]:
from functools import partial
optimisation_func = partial(objective, df=train_folds)

In [ ]:
import optuna
study = optuna.create_study(direction="maximize")
study.optimize(optimisation_func, n_trials=100)

In [ ]:
best_params = study.best_params

In [ ]:
best_params

In [ ]:
new_features = f27_features(test)
test = pd.concat([test, new_features], axis=1)
test.drop(columns=["f_27"], inplace=True)

In [ ]:
test.columns

In [ ]:
x = train_folds.iloc[:, 3:].to_numpy()
y = train_folds["target"].to_numpy()
final_train_set = lightgbm.Dataset(x,y)
params = {
    'objective':'binary',
    'metric': 'binary_error',
    'boosting':'gbdt',
    'n_estimators':1000,
    'num_leaves':best_params["num_leaves"],
    'feature_fraction':best_params["feature_fraction"],
    'bagging_fraction':best_params["bagging_fraction"],
    'learning_rate':best_params["learning_rate"],
    'verbose':1
}
fitted_final_model = lightgbm.train(params, final_train_set)

In [ ]:
test_ = test.iloc[:, 2:].to_numpy()
ypred = fitted_final_model.predict(test_)
'''y_pred = []
for pred in ypred:
    if pred >= 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)
'''

In [ ]:
submission = pd.DataFrame({ 'id': test["id"],
                            'target': ypred})

In [ ]:
submission.to_csv("submission.csv", index=False)